# Confluence Integration Test Notebook

This notebook tests the Confluence integration functionality including:
- Authentication
- Space operations
- Page management (CRUD operations)
- Content search
- Deployment page creation
- Connection validation

In [ ]:
# Setup and imports
import os
import asyncio
import json
from typing import Dict, Any, List
from IPython.display import display, HTML

# Change to backend directory
os.chdir("./")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# Import Confluence integration components
from app.integrations.factory import get_api_factory
from app.integrations.base.confluence_interface import (
    ConfluencePage, ConfluenceSpace
)
from app.core.config import get_settings

print("Confluence integration components imported successfully")

In [ ]:
# Initialize API factory and get Confluence client
factory = get_api_factory()
confluence_client = factory.create_client("confluence", use_mock=True)  # Use mock for testing

print(f"Confluence client created: {type(confluence_client).__name__}")
print(f"Using mock implementation: {hasattr(confluence_client, '_mock_data')}")

## 1. Authentication Test

In [ ]:
# Test authentication
async def test_authentication():
    try:
        auth_result = await confluence_client.authenticate()
        print(f"Authentication result: {auth_result}")
        return auth_result
    except Exception as e:
        print(f"Authentication error: {e}")
        return False

# Run authentication test
auth_success = await test_authentication()
print(f"✅ Authentication test {'PASSED' if auth_success else 'FAILED'}")

## 2. Connection Validation Test

In [ ]:
# Test connection validation
async def test_connection_validation():
    try:
        connection_info = await confluence_client.validate_connection()
        print("Connection validation result:")
        print(json.dumps(connection_info, indent=2))
        return connection_info
    except Exception as e:
        print(f"Connection validation error: {e}")
        return None

# Run connection validation test
connection_info = await test_connection_validation()
print(f"✅ Connection validation test {'PASSED' if connection_info else 'FAILED'}")

## 3. Space Operations Test

In [ ]:
# Test space operations
async def test_space_operations():
    try:
        # Get all spaces
        spaces = await confluence_client.get_spaces()
        print(f"Found {len(spaces)} spaces")
        
        for space in spaces[:5]:  # Show first 5 spaces
            print(f"  - {space.key}: {space.name} ({space.type})")
            if space.description:
                print(f"    Description: {space.description[:100]}...")
        
        if len(spaces) > 5:
            print(f"  ... and {len(spaces) - 5} more spaces")
        
        return True
    except Exception as e:
        print(f"Space operations error: {e}")
        return False

# Run space operations test
space_test_success = await test_space_operations()
print(f"✅ Space operations test {'PASSED' if space_test_success else 'FAILED'}")

## 4. Page CRUD Operations Test

In [ ]:
# Test page CRUD operations
async def test_page_crud_operations():
    test_space_key = "TEST"
    test_page_title = "Test Page"
    test_content = "<p>This is a test page content.</p>"
    
    try:
        # Create a new page
        print("Creating new page...")
        new_page = await confluence_client.create_page(
            space_key=test_space_key,
            title=test_page_title,
            content=test_content
        )
        print(f"Created page: {new_page.title} (ID: {new_page.id})")
        
        # Get the created page
        print("\nRetrieving created page...")
        retrieved_page = await confluence_client.get_page(new_page.id)
        if retrieved_page:
            print(f"Retrieved page: {retrieved_page.title}")
            print(f"  Content length: {len(retrieved_page.content or '')} characters")
            print(f"  Version: {retrieved_page.version}")
        
        # Update the page
        print("\nUpdating page...")
        updated_content = "<p>This is updated test page content.</p>"
        updated_page = await confluence_client.update_page(
            page_id=new_page.id,
            title=f"{test_page_title} (Updated)",
            content=updated_content,
            version=new_page.version
        )
        print(f"Updated page: {updated_page.title} (Version: {updated_page.version})")
        
        # Delete the page
        print("\nDeleting page...")
        delete_result = await confluence_client.delete_page(new_page.id)
        print(f"Delete result: {delete_result}")
        
        return True
    except Exception as e:
        print(f"Page CRUD operations error: {e}")
        return False

# Run page CRUD operations test
page_crud_test_success = await test_page_crud_operations()
print(f"✅ Page CRUD operations test {'PASSED' if page_crud_test_success else 'FAILED'}")

## 5. Page Search Test

In [ ]:
# Test page search operations
async def test_page_search():
    test_space_key = "TEST"
    
    try:
        # Search pages in space
        print("Searching pages in space...")
        pages = await confluence_client.search_pages(test_space_key)
        print(f"Found {len(pages)} pages in space '{test_space_key}'")
        
        for page in pages[:5]:  # Show first 5 pages
            print(f"  - {page.title} (ID: {page.id})")
            print(f"    URL: {page.url}")
            print(f"    Version: {page.version}")
        
        if len(pages) > 5:
            print(f"  ... and {len(pages) - 5} more pages")
        
        # Search pages with specific title
        print("\nSearching pages with specific title...")
        specific_pages = await confluence_client.search_pages(
            test_space_key, title="Test"
        )
        print(f"Found {len(specific_pages)} pages with 'Test' in title")
        
        return True
    except Exception as e:
        print(f"Page search error: {e}")
        return False

# Run page search test
page_search_test_success = await test_page_search()
print(f"✅ Page search test {'PASSED' if page_search_test_success else 'FAILED'}")

## 6. Deployment Page Creation Test

In [ ]:
# Test deployment page creation
async def test_deployment_page_creation():
    test_space_key = "TEST"
    release_version = "v1.0.0"
    test_repositories = [
        {
            "name": "test-repo-1",
            "url": "https://github.com/test/test-repo-1",
            "branch": "main",
            "commit": "abc123def456"
        },
        {
            "name": "test-repo-2",
            "url": "https://github.com/test/test-repo-2",
            "branch": "develop",
            "commit": "def456ghi789"
        }
    ]
    
    try:
        # Create deployment page
        deployment_page = await confluence_client.create_deployment_page(
            space_key=test_space_key,
            release_version=release_version,
            repositories=test_repositories
        )
        
        print(f"Created deployment page: {deployment_page.title}")
        print(f"  Page ID: {deployment_page.id}")
        print(f"  URL: {deployment_page.url}")
        print(f"  Content length: {len(deployment_page.content or '')} characters")
        
        # Display a preview of the content
        if deployment_page.content:
            content_preview = deployment_page.content[:200] + "..." if len(deployment_page.content) > 200 else deployment_page.content
            print(f"\nContent preview:\n{content_preview}")
        
        return True
    except Exception as e:
        print(f"Deployment page creation error: {e}")
        return False

# Run deployment page creation test
deployment_page_test_success = await test_deployment_page_creation()
print(f"✅ Deployment page creation test {'PASSED' if deployment_page_test_success else 'FAILED'}")

## 7. Content Formatting Test

In [ ]:
# Test different content formatting
async def test_content_formatting():
    test_space_key = "TEST"
    
    # Different content formats to test
    content_formats = [
        {
            "name": "HTML Content",
            "content": """
            <h1>Test Page</h1>
            <p>This is a <strong>bold</strong> paragraph with <em>italic</em> text.</p>
            <ul>
                <li>List item 1</li>
                <li>List item 2</li>
            </ul>
            <table border="1">
                <tr><th>Header 1</th><th>Header 2</th></tr>
                <tr><td>Data 1</td><td>Data 2</td></tr>
            </table>
            """
        },
        {
            "name": "Confluence Markup",
            "content": """
            h1. Test Page
            
            This is a *bold* paragraph with _italic_ text.
            
            * List item 1
            * List item 2
            
            ||Header 1||Header 2||
            |Data 1|Data 2|
            """
        },
        {
            "name": "Code Block",
            "content": """
            <h2>Code Example</h2>
            <pre><code class="python">
            def hello_world():
                print("Hello, World!")
                return True
            </code></pre>
            """
        }
    ]
    
    try:
        for i, format_test in enumerate(content_formats, 1):
            print(f"\n--- Testing {format_test['name']} ---")
            
            page = await confluence_client.create_page(
                space_key=test_space_key,
                title=f"Format Test {i}: {format_test['name']}",
                content=format_test['content']
            )
            
            print(f"Created page: {page.title} (ID: {page.id})")
            print(f"Content length: {len(page.content or '')} characters")
            
            # Clean up - delete the test page
            await confluence_client.delete_page(page.id)
            print(f"Deleted test page: {page.id}")
        
        return True
    except Exception as e:
        print(f"Content formatting error: {e}")
        return False

# Run content formatting test
content_format_test_success = await test_content_formatting()
print(f"✅ Content formatting test {'PASSED' if content_format_test_success else 'FAILED'}")

## 8. Parent-Child Page Structure Test

In [ ]:
# Test parent-child page structure
async def test_parent_child_structure():
    test_space_key = "TEST"
    
    try:
        # Create parent page
        print("Creating parent page...")
        parent_page = await confluence_client.create_page(
            space_key=test_space_key,
            title="Parent Page",
            content="<p>This is the parent page.</p>"
        )
        print(f"Created parent page: {parent_page.title} (ID: {parent_page.id})")
        
        # Create child page under parent
        print("\nCreating child page...")
        child_page = await confluence_client.create_page(
            space_key=test_space_key,
            title="Child Page",
            content="<p>This is the child page.</p>",
            parent_id=parent_page.id
        )
        print(f"Created child page: {child_page.title} (ID: {child_page.id})")
        print(f"Parent ID: {parent_page.id}")
        
        # Create grandchild page
        print("\nCreating grandchild page...")
        grandchild_page = await confluence_client.create_page(
            space_key=test_space_key,
            title="Grandchild Page",
            content="<p>This is the grandchild page.</p>",
            parent_id=child_page.id
        )
        print(f"Created grandchild page: {grandchild_page.title} (ID: {grandchild_page.id})")
        print(f"Parent ID: {child_page.id}")
        
        # Clean up - delete pages in reverse order
        print("\nCleaning up pages...")
        await confluence_client.delete_page(grandchild_page.id)
        await confluence_client.delete_page(child_page.id)
        await confluence_client.delete_page(parent_page.id)
        print("All test pages deleted")
        
        return True
    except Exception as e:
        print(f"Parent-child structure error: {e}")
        return False

# Run parent-child structure test
parent_child_test_success = await test_parent_child_structure()
print(f"✅ Parent-child structure test {'PASSED' if parent_child_test_success else 'FAILED'}")

## 9. Real vs Mock Comparison Test

In [ ]:
# Compare mock vs real implementations
async def compare_mock_vs_real():
    print("=== Mock Implementation ===")
    mock_client = factory.create_client("confluence", use_mock=True)
    mock_auth = await mock_client.authenticate()
    print(f"Mock auth: {mock_auth}")
    
    print("\n=== Real Implementation ===")
    try:
        real_client = factory.create_client("confluence", use_mock=False)
        real_auth = await real_client.authenticate()
        print(f"Real auth: {real_auth}")
    except Exception as e:
        print(f"Real implementation error (expected if no real credentials): {e}")

# Run comparison test
await compare_mock_vs_real()

## 10. Error Handling Test

In [ ]:
# Test error handling
async def test_error_handling():
    print("Testing error handling scenarios...")
    
    # Test with invalid page ID
    try:
        await confluence_client.get_page("999999")
    except Exception as e:
        print(f"✅ Expected error for invalid page ID: {type(e).__name__}")
    
    # Test with invalid space key
    try:
        await confluence_client.search_pages("INVALID_SPACE")
    except Exception as e:
        print(f"✅ Expected error for invalid space key: {type(e).__name__}")
    
    # Test with invalid parent ID
    try:
        await confluence_client.create_page(
            space_key="TEST",
            title="Test Page",
            content="<p>Test</p>",
            parent_id="999999"
        )
    except Exception as e:
        print(f"✅ Expected error for invalid parent ID: {type(e).__name__}")
    
    print("Error handling tests completed")

In [ ]:
# Run error handling test
await test_error_handling()

## 11. Performance Test

In [ ]:
# Test performance with multiple concurrent requests
async def test_performance():
    import time
    
    print("Testing performance with concurrent requests...")
    
    # Test single request timing
    start_time = time.time()
    await confluence_client.get_spaces()
    single_request_time = time.time() - start_time
    print(f"Single request time: {single_request_time:.3f}s")
    
    # Test multiple concurrent requests
    start_time = time.time()
    tasks = [
        confluence_client.get_spaces(),
        confluence_client.search_pages("TEST"),
        confluence_client.validate_connection()
    ]
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    concurrent_time = time.time() - start_time
    
    print(f"Concurrent requests time: {concurrent_time:.3f}s")
    print(f"Speedup: {single_request_time * len(tasks) / concurrent_time:.2f}x")
    
    # Check results
    successful_requests = sum(1 for r in results if not isinstance(r, Exception))
    print(f"Successful concurrent requests: {successful_requests}/{len(tasks)}")

# Run performance test
await test_performance()

## 12. Summary and Results

In [ ]:
# Display test summary
def display_test_summary():
    summary = {
        "Authentication": auth_success,
        "Connection Validation": bool(connection_info),
        "Space Operations": space_test_success,
        "Page CRUD Operations": page_crud_test_success,
        "Page Search": page_search_test_success,
        "Deployment Page Creation": deployment_page_test_success,
        "Content Formatting": content_format_test_success,
        "Parent-Child Structure": parent_child_test_success,
    }
    
    print("\n" + "="*50)
    print("CONFLUENCE INTEGRATION TEST SUMMARY")
    print("="*50)
    
    for test_name, result in summary.items():
        status = "✅ PASS" if result else "❌ FAIL"
        print(f"{test_name:<25} {status}")
    
    total_tests = len(summary)
    passed_tests = sum(summary.values())
    print(f"\nTotal Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {total_tests - passed_tests}")
    print(f"Success Rate: {(passed_tests/total_tests)*100:.1f}%")

# Display summary
display_test_summary()